In [74]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

In [75]:
df=pd.read_csv('../data/data2020.csv')
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [76]:
df1 = df.copy()
le=LabelEncoder()
categorical = df.select_dtypes(include = 'O')
categorical.columns
for feature in categorical:
    df1[feature]=le.fit_transform(df1[feature])
    print(feature)

HeartDisease
Smoking
AlcoholDrinking
Stroke
DiffWalking
Sex
AgeCategory
Race
Diabetic
PhysicalActivity
GenHealth
Asthma
KidneyDisease
SkinCancer


In [4]:
def split_data(X, Y):
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,stratify=Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,stratify=df1['HeartDisease'])
    return X_train, X_test, y_train, y_test
def over_sample_date(X_train, y_train, data_rate = 1):
    minority_class = y_train.value_counts().idxmin()    # 1
    majority_class = y_train.value_counts().idxmax()    # 0

    minority_data = X_train[y_train == minority_class]
    majority_data = X_train[y_train == majority_class]
    minority_labels = y_train[y_train == minority_class]
    majority_labels = y_train[y_train == majority_class]

    oversampled_minority_data = minority_data.sample(int(len(majority_data)*data_rate), replace=True)
    oversampled_minority_labels = minority_labels.sample(int(len(majority_data)*data_rate), replace=True)
    X_train_oversampled = pd.concat([majority_data, oversampled_minority_data])
    y_train_oversampled = pd.concat([majority_labels, oversampled_minority_labels])
    return X_train_oversampled, y_train_oversampled

## Change data_rate and other things to improve the data

In [77]:
# get the training dataset and test dataset
X=df1.drop('HeartDisease',axis=1)
Y=df1['HeartDisease']
X_train, X_test, y_train, y_test = split_data(X,Y)
X_train_oversampled, Y_train_oversampled = over_sample_date(X_train, y_train, data_rate = 1)

In [78]:
# transform the training data
scaler = StandardScaler()
X_train_oversampled_T = scaler.fit_transform(X_train_oversampled)
X_test_T = scaler.transform(X_test)

In [79]:
log_reg = LogisticRegression()
log_reg.fit(X_train_oversampled_T, Y_train_oversampled)

y_pred = log_reg.predict(X_test_T)
#acc=log_reg.score(y_test,y_pred)
print(classification_report(y_test, y_pred))
accuracy=log_reg.score(X_test,y_test)
print(accuracy)
confusion_matrix(y_test,y_pred)

              precision    recall  f1-score   support

           0       0.97      0.74      0.84     58484
           1       0.22      0.76      0.34      5475

    accuracy                           0.74     63959
   macro avg       0.59      0.75      0.59     63959
weighted avg       0.91      0.74      0.80     63959

0.08560171359777358


/Users/pb/anaconda3/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([[43346, 15138],
       [ 1296,  4179]])

In [83]:
log_reg.coef_,log_reg.intercept_

(array([[ 0.11093823,  0.23342849, -0.05418123,  0.3869928 ,  0.25568301,
          0.11938427,  0.1761645 ,  0.37207277,  1.02793546,  0.02749455,
          0.26370807, -0.04490726, -0.03153452, -0.04868793,  0.13806599,
          0.19823492,  0.04858497]]),
 array([-0.0196599]))

In [151]:
from sklearn.preprocessing import StandardScaler

def normalize_row(row, scaler):
    # Reshape row to match expected input of scaler
    row_reshaped = row.values.reshape(1, -1)
    # Apply the same transformation as training data
    normalized_row = scaler.transform(row_reshaped)
    return normalized_row  # Flatten to match the shape of the original row

# Assuming 'scaler' is already fitted to 'X_train_oversampled'
scaler = StandardScaler().fit(X_train_oversampled)

df_new = X_train_oversampled.iloc[0]

df_new_normalized = normalize_row(df_new, scaler)

print(df_new_normalized)
print(X_train_oversampled_T[0] == df_new_normalized)


[[-0.72063279  1.01757354 -0.24489727 -0.32061579 -0.44128201 -0.49706031
  -0.56345196  0.94606575 -1.70156519 -1.25708171 -0.56441101  0.63194933
   1.30354385 -0.69905489 -0.42729395 -0.28913775 -0.39388304]]
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True]]


/Users/pb/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [162]:
# Use user input to get 
import numpy as np

user_input = [21.93, 0, 0.0, 0.0, 20.0, 0.0, 1.0, 1.0, 9.0, 5.0, 2.0, 0.0, 1.0, 3.0, 0.0, 1.0, 1.0]

user_input_T = normalize_row(pd.DataFrame([user_input]),scaler)
coef = np.array([0.11093823, 0.23342849, -0.05418123, 0.3869928, 0.25568301,
                 0.11938427, 0.1761645, 0.37207277, 1.02793546, 0.02749455,
                 0.26370807, -0.04490726, -0.03153452, -0.04868793, 0.13806599,
                 0.19823492, 0.04858497]).flatten()
intercept = -0.0196599

#user_input_T = np.array([-0.72063279, 1.01757354, -0.24489727, -0.32061579, -0.44128201, -0.49706031,
#              -0.56345196, 0.94606575, -1.70156519, -1.25708171, -0.56441101, 0.63194933,
#              1.30354385, -0.69905489, -0.42729395, -0.28913775, -0.39388304])

# Compute the linear combination of inputs and weights
z = np.dot(user_input_T, coef) + intercept

# Apply the logistic function
p = 1 / (1 + np.exp(-z))

print("The predicted probability is:", p)


The predicted probability is: [0.91205902]


/Users/pb/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [127]:
X_train_oversampled.iloc[30]
for i in range(1000):
    if Y_train_oversampled[i] == 1:
        print(i)
        break

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [137]:
print(list(X_train_oversampled.iloc[258229]))

[31.93, 1.0, 0.0, 0.0, 20.0, 0.0, 1.0, 1.0, 9.0, 5.0, 2.0, 0.0, 1.0, 3.0, 0.0, 1.0, 1.0]
